In [ ]:
import os
import csv
import pandas as pd
from skimage import io #pip install scikit-image

def list_files_in_folder(folder_path):
    filenames = os.listdir(folder_path)
    return filenames

def write_filenames_to_csv(filenames, parameterized_string, csv_file_path):
    with open(csv_file_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Filename', 'Parameterized String'])  # header row
        for filename in filenames:
            writer.writerow([filename, parameterized_string])
            
def get_all_dir_paths(start_dir):
    paths = []
    for root, dirs, files in os.walk(start_dir):
        for dir in dirs:
            full_path = os.path.join(root, dir)
            paths.append(full_path)
    return paths

def image_to_array(image_path):
    img = io.imread(image_path)
    return img

def label_images(start_dir):
    dir_paths = get_all_dir_paths(start_dir)
    data = []
    for folder_path in dir_paths:
        label = folder_path.split("\\")[7]
        filenames = list_files_in_folder(folder_path)
        for filename in filenames:
            data.append({'filename': filename, 'label': label, 'image': image_to_array(folder_path+'/'+filename)})
    return data

In [50]:
# ### Push Filenames and lables into PANDAS DF
# lables = []
# for folder_path in paths:
#     label = (folder_path.split("\\")[7])
#     filenames = list_files_in_folder(folder_path)
#     write_filenames_to_csv(filenames, label, csv_file_path)

In [31]:
start_dir = r"C:\Users\Manideep S\OneDrive - COGNINE\ML\AnimalHQ\train"
csv_file_path = 'lables.csv'
lables = []
data = label_images(start_dir)
df = pd.DataFrame(data)
print(df)
print(df["label"].unique())

In [21]:
print(df.image[1].shape)

(512, 512, 3)


In [25]:
type(df.image[1])

numpy.ndarray

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# def normalize_image(image):
#     return image / 255.0

# train_images = train_df['image'].apply(normalize_image)
# test_images = test_df['image'].apply(normalize_image)

In [27]:
import dask.dataframe as dd #pip install dask[complete]

# Convert your Pandas DataFrames to Dask DataFrames
train_ddf = dd.from_pandas(train_df, npartitions=8)  # adjust npartitions based on your dataset size
test_ddf = dd.from_pandas(test_df, npartitions=8)

# Define the normalize_image function
def normalize_image(image):
    # your image normalization code here
    return (image / 255.0)

# Perform batch processing using map_partitions
train_images = train_ddf['image'].map_partitions(normalize_image).compute()
test_images = test_ddf['image'].map_partitions(normalize_image).compute()

ValueError: Metadata inference failed in `normalize_image`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
ArrowNotImplementedError("Function 'divide' has no kernel matching input types (large_string, double)")

Traceback:
---------
  File "d:\anaconda\envs\myLLM\Lib\site-packages\dask\dataframe\utils.py", line 195, in raise_on_meta_error
    yield
  File "d:\anaconda\envs\myLLM\Lib\site-packages\dask_expr\_expr.py", line 3988, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Manideep S\AppData\Local\Temp\ipykernel_33408\4202041004.py", line 10, in normalize_image
    return (image / 255.0)
            ~~~~~~^~~~~~~
  

In [ ]:
train_labels = pd.get_dummies(train_df['label']).values
test_labels = pd.get_dummies(test_df['label']).values

In [ ]:
image_shape = (512, 512, 3)  # adjust to your image shape
train_images = train_images.values.reshape(-1, *image_shape)
test_images = test_images.values.reshape(-1, *image_shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=image_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(train_labels.shape[1], activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))

In [ ]:
loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {accuracy:.3f}')